In [1]:
!git clone https://github.com/akshay5305/CNN-vs-VIT-for-Medical-Imaging.git

Cloning into 'CNN-vs-VIT-for-Medical-Imaging'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 23 (delta 3), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 8.58 KiB | 8.58 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
%cd CNN-vs-VIT-for-Medical-Imaging


/content/CNN-vs-VIT-for-Medical-Imaging


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vuppalaadithyasairam/bone-fracture-detection-using-xrays")

print("Path to dataset files:", path)

100%|██████████| 172M/172M [00:01<00:00, 95.1MB/s]

Extracting files...


KeyboardInterrupt: 

In [8]:
import os

BASE_PATH = "/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val"

for root, dirs, files in os.walk(BASE_PATH):
    print(root, "->", dirs)
    break


In [5]:
import tensorflow as tf


from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD


from keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from utils import config as cfg
from utils.utils import return_classes


In [6]:
def return_ds(input_dir):
    train_ds=tf.keras.preprocessing.image_dataset_from_directory(input_dir,
    validation_split=0.2,
     subset='training',
     seed=42,
     image_size=(cfg.input_dim,cfg.input_dim),
     batch_size=16,
     label_mode='categorical',)

    val_ds=tf.keras.preprocessing.image_dataset_from_directory(
        input_dir,
        validation_split=0.2,
        subset='validation',
        seed=42,
        image_size=(cfg.input_dim,cfg.input_dim),
        batch_size=8,
        label_mode='categorical',)

    return train_ds, val_ds


In [2]:
def return_model(input_dim, nb_classes, freeze=False, head=None):
    if head=='xception' or head=='Xception':
        base_model=Xception(include_top=False,weights='imagenet',input_shape=(input_dim,input_dim,3))

    if head=='vg16' or head=='VGG16':
        base_model=VGG16(include_top=False, weights='imagenet',input_shape=(input_dim,input_dim,3))

    if head=='inceptionv3' or 'Inceptionv3' or 'InceptionV3':
        base_model=InceptionV3(include_top=False, weights='imagenet',input_shape=(input_dim,input_dim,3))

    if head=='densenet121':
        base_model=DenseNet121(include_top=False, weights='imagenet',input_shape=(input_dim,input_dim,3))

    if not head:
        print('Please choose pre trained model')

    print('Choose smodel is ', head)

    if freeze:
        for layer in base_model.layers:
            layer.trainable=False

    x=base_model.output
    x=GlobalAveragePooling2D()(x)

    predictions=Dense(nb_classes,activation='softmax')(x)
    model=Model(inputs=base_model.inputs, outputs=predictions)

    return model


In [8]:
if __name__=='__main__':
    classes=return_classes('utils/annotation.txt')
    train_ds,val_ds=return_ds('/kaggle/input/bone-fracture-detection-using-xrays/archive (6)')
    model=return_model(cfg.input_dim,len(classes),head=cfg.head,freeze=True)


    model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=cfg.lr),
    metrics=['accuracy']
)







    save_weights = ModelCheckpoint(filepath='models/xception_freeze_model_40_epoch.h5', monitor='val_accuracy',
                                    verbose=1, save_best_only=True, save_weights_only=False, mode='max')

    model.fit(train_ds, epochs=cfg.epochs, validation_data=val_ds, callbacks=[save_weights])



Found 9463 files belonging to 2 classes.
Using 7571 files for training.
Found 9463 files belonging to 2 classes.
Using 1892 files for validation.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Choose smodel is  xception
Epoch 1/40
474/474 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.8884 - loss: 145.2904
Epoch 1: val_accuracy improved from -inf to 0.06395, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 77s 128ms/step - accuracy: 0.8885 - loss: 145.2259 - val_accuracy: 0.0640 - val_loss: 2358.8276
Epoch 2/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9012 - loss: 126.2256
Epoch 2: val_accuracy improved from 0.06395 to 0.93605, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.9013 - loss: 126.0754 - val_accuracy: 0.9360 - val_loss: 143.7764
Epoch 3/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9218 - loss: 59.8636
Epoch 3: val_accuracy improved from 0.93605 to 0.94450, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.9217 - loss: 59.8503 - val_accuracy: 0.9445 - val_loss: 14.2787
Epoch 4/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9183 - loss: 51.8705
Epoch 4: val_accuracy did not improve from 0.94450
474/474 ━━━━━━━━━━━━━━━━━━━━ 40s 84ms/step - accuracy: 0.9183 - loss: 51.8499 - val_accuracy: 0.9403 - val_loss: 35.9583
Epoch 5/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9367 - loss: 34.7190
Epoch 5: val_accuracy did not improve from 0.94450
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9367 - loss: 34.7365 - val_accuracy: 0.1337 - val_loss: 971.5078
Epoch 6/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9162 - loss: 87.8674
Epoch 6: val_accuracy did not improve from 0.94450
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9163 - loss: 87.7109 - val_accuracy: 0.9392 - val_loss: 47.3746
Epoch 7/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9394 - loss: 30.3343
Epoch 7: v

474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.9394 - loss: 30.3298 - val_accuracy: 0.9577 - val_loss: 8.3006
Epoch 8/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9379 - loss: 31.6220
Epoch 8: val_accuracy did not improve from 0.95772
474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 81ms/step - accuracy: 0.9379 - loss: 31.6037 - val_accuracy: 0.9567 - val_loss: 16.4532
Epoch 9/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9391 - loss: 33.9092
Epoch 9: val_accuracy did not improve from 0.95772
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9392 - loss: 33.8967 - val_accuracy: 0.8943 - val_loss: 25.4597
Epoch 10/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9411 - loss: 32.2523
Epoch 10: val_accuracy did not improve from 0.95772
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9411 - loss: 32.2441 - val_accuracy: 0.9572 - val_loss: 13.6683
Epoch 11/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9405 - loss: 30.2602
Epoch 11:

...

474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 81ms/step - accuracy: 0.9566 - loss: 18.1840 - val_accuracy: 0.9794 - val_loss: 4.6692
Epoch 20/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9547 - loss: 20.1906
Epoch 20: val_accuracy did not improve from 0.97939
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9547 - loss: 20.1862 - val_accuracy: 0.9757 - val_loss: 5.4883
Epoch 21/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9642 - loss: 12.2188
Epoch 21: val_accuracy improved from 0.97939 to 0.98044, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.9642 - loss: 12.2078 - val_accuracy: 0.9804 - val_loss: 3.7799
Epoch 22/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9566 - loss: 18.0303
Epoch 22: val_accuracy did not improve from 0.98044
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9567 - loss: 18.0108 - val_accuracy: 0.9804 - val_loss: 5.0858
Epoch 23/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9534 - loss: 19.5262
Epoch 23: val_accuracy did not improve from 0.98044
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 80ms/step - accuracy: 0.9535 - loss: 19.5159 - val_accuracy: 0.9762 - val_loss: 4.2820
Epoch 24/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9608 - loss: 12.9323
Epoch 24: val_accuracy did not improve from 0.98044
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9608 - loss: 12.9176 - val_accuracy: 0.9746 - val_loss: 5.6587
Epoch 25/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9574 - loss: 20.9928
Epoch 25

474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.9653 - loss: 11.5565 - val_accuracy: 0.9831 - val_loss: 3.6837
Epoch 29/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9656 - loss: 15.9604
Epoch 29: val_accuracy did not improve from 0.98309
474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.9656 - loss: 15.9363 - val_accuracy: 0.9831 - val_loss: 3.6129
Epoch 30/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9707 - loss: 9.9024
Epoch 30: val_accuracy improved from 0.98309 to 0.98362, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 83ms/step - accuracy: 0.9707 - loss: 9.8936 - val_accuracy: 0.9836 - val_loss: 3.4059
Epoch 31/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9675 - loss: 10.2750
Epoch 31: val_accuracy improved from 0.98362 to 0.98731, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.9675 - loss: 10.2683 - val_accuracy: 0.9873 - val_loss: 2.8078
Epoch 32/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9642 - loss: 12.0973
Epoch 32: val_accuracy did not improve from 0.98731
474/474 ━━━━━━━━━━━━━━━━━━━━ 40s 81ms/step - accuracy: 0.9642 - loss: 12.0855 - val_accuracy: 0.9540 - val_loss: 18.0075
Epoch 33/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9632 - loss: 13.0890
Epoch 33: val_accuracy did not improve from 0.98731
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9632 - loss: 13.0778 - val_accuracy: 0.9360 - val_loss: 59.1495
Epoch 34/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9682 - loss: 12.7666
Epoch 34: val_accuracy did not improve from 0.98731
474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.9682 - loss: 12.7477 - val_accuracy: 0.9778 - val_loss: 4.0579
Epoch 35/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9699 - loss: 8.3250
Epoch 3

In [4]:
from keras.models import load_model
from PIL import Image
from keras.preprocessing import image
import numpy as np
import tensorflow as tf



In [5]:
def return_test_ds(input_dir):
    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        input_dir,
        seed=42,
        image_size=(cfg.input_dim, cfg.input_dim),
        batch_size=16,
    label_mode='categorical',
    )
    return test_ds

In [6]:
def predict_one_image(img_path, model_path):
    classes = return_classes(cfg.classes_path)
    model = load_model(model_path)
    img = image.load_img(img_path, target_size=(cfg.input_dim, cfg.input_dim))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_batch)
    predicted_class_idx = np.argmax(prediction)
    probability = np.max(prediction)
    predicted_class = classes[predicted_class_idx]

    return predicted_class, probability

In [7]:
def evaluate_model(model_path):
    test_ds = return_test_ds(cfg.test_dataset_path)
    model = load_model(model_path)
    score = model.evaluate(test_ds, verbose=0)

    return score[0], score[1]  # loss, accuracy